In [38]:
# Imports

import numpy as np
import pandas as pd
import torch
from torch.nn import *
from torch import arange as torch_arange
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from random import shuffle
from torch.utils.data import DataLoader, TensorDataset

# Read and Preprocess the Data

In [57]:
# Read the data
data_folder = os.path.join("data","landscape Images")
gray_folder = os.path.join(data_folder,"gray")
color_folder = os.path.join(data_folder,"color")
data=[]
for file in tqdm(set(os.listdir(gray_folder)).intersection(set(os.listdir(color_folder))),"Reading data images"):
    gray_image = plt.imread(os.path.join(gray_folder,file))/255
    color_image = plt.imread(os.path.join(color_folder,file))/255
    data.append([file,gray_image,color_image])
print(f"The size of the data is {len(data)}")

Reading data images: 100%|██████████| 7129/7129 [00:19<00:00, 361.55it/s]

The size of the data is 7129


In [59]:
test_size = 0.2
shuffle(data)
train_data= data[:int(len(data)*(1-test_size))]
test_data = data[int(len(data)*(1-test_size)):]
print(f"Size of train data: {len(train_data)}")
print(f"Size of test data: {len(test_data)}")

Size of train data: 5703
Size of test data: 1426


In [60]:
train_data[0]

['3380.jpg',
 array([[0.62352941, 0.62352941, 0.62352941, ..., 0.6       , 0.59215686,
         0.58431373],
        [0.67058824, 0.68235294, 0.6745098 , ..., 0.58039216, 0.57647059,
         0.57254902],
        [0.6627451 , 0.69019608, 0.68235294, ..., 0.58431373, 0.58431373,
         0.58823529],
        ...,
        [0.10980392, 0.16078431, 0.24705882, ..., 0.89803922, 0.90196078,
         0.90196078],
        [0.17254902, 0.14509804, 0.00392157, ..., 0.89411765, 0.89803922,
         0.90196078],
        [0.18039216, 0.1372549 , 0.05490196, ..., 0.89019608, 0.89803922,
         0.90588235]]),
 array([[[0.55294118, 0.62745098, 0.78431373],
         [0.55294118, 0.62745098, 0.78431373],
         [0.55294118, 0.62745098, 0.78431373],
         ...,
         [0.50196078, 0.61176471, 0.8       ],
         [0.49803922, 0.6       , 0.79215686],
         [0.49019608, 0.59215686, 0.78431373]],
 
        [[0.6       , 0.6745098 , 0.83137255],
         [0.61176471, 0.68627451, 0.84313725],
   

# Design Model

In [94]:
class ImageColorizerCNNAutoEncoder(Module):
    def __init__(self):
        super(ImageColorizerCNNAutoEncoder,self).__init__()
        self.encoder = Sequential()
        self.encoder.append(Conv2d(in_channels=1,out_channels=64,kernel_size=(3,3),padding="same"))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=64,out_channels=64,kernel_size=(3,3),stride=2))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=64,out_channels=128,kernel_size=(3,3),padding="same"))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=128,out_channels=128,kernel_size=(3,3),stride=2))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=128,out_channels=256,kernel_size=(3,3),padding="same"))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=256,out_channels=256,kernel_size=(3,3),stride=2))
        self.encoder.append(ReLU())
        self.encoder.append(Conv2d(in_channels=256,out_channels=64,kernel_size=(3,3),padding="same"))
        self.encoder.append(ReLU())
        
        self.decoder = Sequential()
        self.decoder.append(Upsample(scale_factor=2))
        self.decoder.append(Conv2d(in_channels=64,out_channels=32, kernel_size=(3,3),padding="same"))
        self.decoder.append(ReLU())
        self.decoder.append(Upsample(scale_factor=2))
        self.decoder.append(Conv2d(in_channels=32,out_channels=16, kernel_size=(3,3),padding="same"))
        self.decoder.append(ReLU())
        self.decoder.append(Upsample(scale_factor=2))
        self.decoder.append(Conv2d(in_channels=16,out_channels=3, kernel_size=(3,3),padding="same"))
        self.decoder.append(Sigmoid())        
    def forward(self,x):
        x= torch.tensor(x,dtype=torch.float).unsqueeze(0)
        print(self.encoder(x).shape)
        return self.decoder(self.encoder(x))
cnn = ImageColorizerCNNAutoEncoder()

cnn(data[0][1]).shape

torch.Size([64, 17, 17])


torch.Size([3, 17, 136])

In [95]:
data[0][1].shape

(150, 150)

In [93]:
for module in cnn.modules():
    print(module)

ImageColorizerCNNAutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
    (7): ReLU()
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (9): ReLU()
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
    (11): ReLU()
    (12): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (13): ReLU()
  )
  (decoder): Sequential(
    (0): Upsample(scale_factor=2.0, mode='nearest')
    (1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): ReLU()
    (3): Upsample(scale_factor=2.0, mode='nearest')
    (4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): Upsample(scale_factor=2.0, mode='

In [76]:
plt.imread(os.path.join(gray_folder,"2778.jpg")).shape

(100, 150)

# Train Model

In [48]:
# Hyper parameters
learning_rate = 0.1
num_epochs = 4
batch_size = 32

in_features = data[0][1].size
out_features = data[0][2].size
hidden_layers = []
print('0')
dnn = ImageColorizerDNN(in_features, out_features, hidden_layers)
print('1')
optimizer = torch.optim.Adam(dnn.parameters(),lr=learning_rate)
criterion = MSELoss()
print('2')
train_dataset = TensorDataset(torch.tensor(np.array([d[1] for d in train_data])),torch.tensor(np.array([d[2] for d in train_data])))
train_loader =  DataLoader(train_dataset,batch_size=batch_size)
print('3')
for i in range(num_epochs):
    epoch_loss = 0
    for inputs, labels in train_loader:
        print(len(inputs))
        break
    break


0
1
2


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5703,) + inhomogeneous part.

In [52]:
np.unique(np.array([len(d[1]) for d in train_data]))

array([11400, 12150, 15000, 16200, 16500, 16650, 16950, 18600, 19650,
       20400, 21150, 21450, 21750, 21900, 22500])

In [47]:
train_data[0][1].size

22500